In [5]:
import SimpleITK as sitk
img = sitk.ReadImage('./wd0000_ps_image.nii.gz') #size = 512x512x96, spacing = 1x1x1


print("Image Size: ", img.GetSize())
print("Image Spacing: ", img.GetSpacing())


Image Size:  (512, 512, 254)
Image Spacing:  (0.798828125, 0.798828125, 2.0)


In [6]:
ref = sitk.Image(96, 512,512, sitk.sitkFloat32)
ref.SetOrigin(img.GetOrigin())
ref.SetDirection(img.GetDirection())
spacing = img.GetSpacing() # x,y,z spacing
size = img.GetSize()
spacing_x = spacing[0]*size[0]/96
spacing_y = spacing[1]*size[1]/512
spacing_z = spacing[2]*size[2]/512

ref.SetSpacing((spacing_x,spacing_y,spacing_z))

In [7]:
res = sitk.ResampleImageFilter()
res.SetReferenceImage(ref)
img_resampled = res.Execute(img)

In [8]:
sitk.WriteImage(img_resampled, './resampled.nii.gz')